In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import re
import copy

In [2]:
path = 'D:/AS/'

In [4]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [5]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']=='AS']

In [6]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')

In [7]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')

In [8]:
vt_tax_based_on = pd.read_csv(r'D:\AS\vt_tax_based_on.csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\AS\vt_tax_based_on_permit_info.csv')

In [9]:
vm_vh_class=pd.read_csv(r'D:\Jharkhand python\vm_vh_class.csv')

In [3]:
vt_owner = pd.read_csv(r'D:\AS\vt_owner.csv')

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
set(list(vt_owner['P Add1']))

{nan,
 'VILL-RONGMEI NAGA PUNJI',
 'VILL- GUPALPUR GAON',
 'H/NO-11 HILL AVENUE NR BY SUBHAM',
 'NAYAPARA W-NO-IV PO-ABHAYAPURI',
 'P K BISWAS KAHILIPARA',
 'VILL   PUB  BANDARMELA',
 'G S RD BHANGGARH',
 'G N B ROAD BARA BAZAR',
 'VILL-DIGHALKUCHI BAMUNPARA',
 'HOUSE NO. - 34, VIP ROAD,',
 'JALDOBA PT-2 PAGLAHAT KAZIGAON OKJ',
 'BASISTHANAGAR MANSHA MANDIR',
 'VILL-BONGAIGAON TOWN W/NO-02',
 'HN-167 NARENGI PATHARQUARY',
 'VILL-PALANGDI HATI W/NO-09',
 'C/O ABDUL HUSSAIN',
 'VILL-(A) GARGAON NO-2',
 'VILL-KHONIKAR GAON',
 'PREMA NAGAR',
 'KATPUHA PO-ARIKUCHI',
 'H/NO-55 BAPUJI NAGAR 2NO MATHGHARIA',
 'COLE ROAD AKBAR PATTY',
 '147 BATTALION C R P F',
 'VILL-UJJANKUPA PT-1',
 'DANGAGAON (SUDEM NWGWR)',
 'NARIKALA-RABHAPARA PO-ABHAYAPURI',
 'VILL- KALAIN CHEARA GRANT',
 'MARANGABARI W/N-55  SONAPUR',
 'VILL- JHORNARCHAR PT-IV',
 'HOUSE NO-23,BYE LANE-8',
 'VILL- GOALPARA TOWN, W/NO-6',
 'KENDUGURI CHENGELI GAON (PART)',
 'VILL MADARGURI',
 'NAJIRA TOWN',
 'C/O- MINTU DAS  LT RABI DAS',


In [11]:
vt_permit = pd.read_csv(r'D:\AS\vt_permit.csv')

In [12]:
tm_office = pd.read_csv(r'D:\AS\tm_office.csv')

In [13]:
vt_tax_exem = pd.read_csv(r'D:\AS\vt_tax_exem.csv')

In [12]:
vt_tax = pd.read_csv(r'D:\AS\vt_tax.csv')
vt_tax = vt_tax[vt_tax['Pur Cd']==58]
vt_tax = vt_tax[vt_tax['Tax Mode']!='B']
#vt_tax=vt_tax[vt_tax['Collected By']!='VAHAN2']
vt_tax.reset_index(drop=True, inplace=True)

In [14]:
#Turn into first letter capital in each word
def change_col_names(cols):
    pp=[]
    for j in cols :
        p= j.replace('_',' ').title()
        pp.append(p)
        #df.rename(columns={j: p},inplace=True)
    return pp

In [15]:
#def_list = pd.read_csv(r'D:\AS\New folder\Defaulters Summary Maxi Cab Kamrup Assam.csv')
#def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\kamrup - Maxi Cab as on 31st July 2019.xls')
#def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Tax Defaulter Report (1).xls')
#def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
#def_list = def_list[(def_list['Office']!='KAMRUP(AS-1)') & (def_list['Office']!='DIBRUGARH(AS-6)')]
#def_list = def_list[def_list['Vehicle Class']=='Motor Cab']
#def_list = def_list[def_list['Tax Arrear From'].notnull()]
#def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
def_list.columns

Index(['Unnamed: 0', 'Regn No', 'Off Cd', 'Vh Class', 'Tax Amt', 'Tax Fine',
       'Max Upto', 'Eng No', 'Fuel', 'Imported Vch', 'Ld Wt', 'Maker',
       'Maker Model', 'Owner Cd', 'Purchase Dt', 'Regn Dt', 'Regn No-1',
       'Seat Cap', 'Sleeper Cap', 'Stand Cap', 'Status', 'Unld Wt',
       'Vch Catg'],
      dtype='object')

In [16]:
def_list = def_list.drop_duplicates(subset=None, keep='first', inplace=False)
#def_list = def_list[def_list['Vh Class']==7]
#def_list = def_list[def_list['Office']=='KAMRUP(AS-1)']
#def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
#def_list = def_list[def_list['Tax Type']=='MV Tax']
def_list.shape

(943, 23)

In [17]:
def_list['Max Upto'] = def_list['Max Upto'].astype('datetime64[ns]')
curr_dt = datetime.strptime('31-Jul-2019','%d-%b-%Y')
def_list = def_list[def_list['Max Upto']<curr_dt]
def_list.shape

(943, 23)

In [18]:
def_list.head()

,Unnamed: 0,Regn No,Off Cd,Vh Class,Tax Amt,Tax Fine,Max Upto,Eng No,Fuel,Imported Vch,...,Owner Cd,Purchase Dt,Regn Dt,Regn No-1,Seat Cap,Sleeper Cap,Stand Cap,Status,Unld Wt,Vch Catg
0,0,AS01BC6364,1,70,72200.0,17365,2010-03-31,486SP27MWZ838687,2.0,N,...,1.0,25-06-2003,25-06-2003,AS01BC6364,10.0,NaN,NaN,A,0.0,OTH
1,1,AS01BC7049,1,70,24700.0,5490,2016-09-30,497SP27AUZ805859,2.0,N,...,1.0,12-05-2006,12-05-2006,AS01BC7049,10.0,NaN,NaN,A,1650.0,OTH
2,2,AS01BC7037,1,70,20900.0,5030,2016-12-31,497SP28KUZ945125,2.0,N,...,1.0,31-12-2006,31-12-2006,AS01BC7037,10.0,NaN,NaN,A,1750.0,OTH
3,3,AS01CC4329,1,70,39900.0,9605,2014-06-30,497SP28CUZ828882,2.0,N,...,1.0,21-10-2005,21-10-2005,AS01CC4329,10.0,NaN,NaN,A,2010.0,OTH
4,4,AS01BC7007,1,70,20900.0,5030,2016-12-31,497SP28MUZ912065,2.0,N,...,1.0,04-02-2007,04-02-2007,AS01BC7007,10.0,NaN,NaN,A,1530.0,OTH


In [20]:
def_list['Vh Class'].value_counts()

70    943
Name: Vh Class, dtype: int64

In [81]:
def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Kamrup DTO Defaulter list up to March 2018.xls')

In [23]:
def_list = def_list[def_list['Vehicle Class']=='Motor Car']
def_list = def_list[def_list['Tax Arrear From'].notnull()]
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]

In [24]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            #print(c_formula,str(m),val)
            c_formula = c_formula.replace(str(m),val)
            #print(c_formula,str(m),val)
        else:
            c_formula = c_formula.replace('<>','!=')

    # Change '=' to '=='
    if '<=' in c_formula:
        c_formula=c_formula.replace('<=','@')
    if '>=' in c_formula:
        c_formula=c_formula.replace('>=',';')
    if '!=' in c_formula:
        c_formula=c_formula.replace('!=','?')
    if '=' in c_formula:
        c_formula = c_formula.replace('=','==')
    if '@' in c_formula:
        c_formula=c_formula.replace('@','<=')
    if ';' in c_formula:
        c_formula=c_formula.replace(';','>=')
    if '?' in c_formula:
        c_formula = c_formula.replace('?','!=')
    return c_formula

In [25]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%d-%m-%Y'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [26]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,new_vch,rcpt_dt):
    for pi in range(len(param_info_owner)):
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        col= col.replace('_',' ').title()
        p_value = ''
        #print(col,par,code,tab)
        #print(datetime.strptime(ot_df['Purchase Dt'].to_list()[0]))
        if par=='vch_age':
            p_value = relativedelta(rcpt_dt, datetime.strptime(ot_df['Purchase Dt'].to_list()[0],date_format))
            p_value = p_value.years
        elif par=='vch_age_months':
            p_value = relativedelta(rcpt_dt, datetime.strptime(ot_df['Purchase Dt'].to_list()[0],date_format))
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age':
            p_value = relativedelta(rcpt_dt, datetime.strptime(ot_df['Purchase Dt'].to_list()[0],date_format))
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        #elif par=='distance_run_in_month':
        #    p_value = vt_tax_based_on['distance_run_in_quarter']/4
        #elif par=='transaction_pur_cd':
        #    appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
        #    if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
        #        print(i,'No pur_cd available in va_details')
        #        print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        #        continue
        #    else:
        #        p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
        #print(p_value)
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['vh_class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = new_vch
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='delay_days':
            #print(rcpt_dt[:10],' ',tax_from)
            p_value = rcpt_dt-tax_from
            #p_value = p_value.years * 365 + p_value.months * 30 + p_value.days
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
            #print(p_value)
        else:
            if tab=='own':
                col= col.replace('_',' ').title()
            p_value= ot_df[col].to_list()[0]
            #print(p_value)
        #print(col,' ',par,' ',code,' ',p_value)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
        #print(code,p_value)
    return params_info

In [27]:
def extract_parameters(regn_no,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['Regn No']==regn_no]
        owner_df= copy.deepcopy(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            for j in c :
                p= j.replace('_',' ').title()
                tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]

    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        #print('TBO')
        tab = 'tbo'
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,new_vch,rcpt_dt)
        #print('TBO',params_info['P_value'].to_list())
    else:
        #print('Owner')
        tab='own'
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,new_vch,rcpt_dt)
    #print('Shape',params_info.shape)
    if len(param_info_other)>0:  
        #print('Other')
        #print(param_info_other.shape)
        for pi in range(len(param_info_other)):
            col = param_info_other['column_name'].to_list()[pi][1:-2]
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            #print(code,col,par)
            
            #print(null_df)
            if math.isnan(params_info[params_info['code']==code]['P_value'].to_list()[0]):
                if col == 'domain_cd':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Domain Cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Domain Cd Available')
                elif col == 'pmt_type':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Pmt Type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Permit Type Available')
                elif col== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                #print(p_value)

                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value  
    return params_info

In [42]:
date_format = "%d-%b-%Y"
def calculate_tax(regn_no,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Y' ,state_cd='AS',off_cd=1):
    rcpt_dt = tax_upto
    if len(vt_owner[vt_owner['Regn No']==regn_no])>0:
        vh_class = vt_owner[vt_owner['Regn No']==regn_no]['Vh Class'].to_list()[0]
        new_vch = 'N'
        #if pur_cd==58:
        #print(vh_class)
    else:
        print(i,'NO INFO IN VT_OWNER',regn_no)
    print(vh_class, pur_cd, tax_mode, tax_from)    
    slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<tax_from) & (slab_new['date_to']>tax_from) ]['slab_code'].to_list()
    print(pur_cd,'Slab Codes Applicable:',slab_codes)
    if len(slab_codes)==0:
        print(i,'Missed',regn_no,'vh_class',vh_class)
        return 0

    formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
        
    #Identify the parameters required for calculating the formulas
    params = []
    for j in range(len(formula_df)):
        formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
        for k in re.split(r'[()\*\s]\s*',formula):
            if '<' in k and '>' in k:
                params.append(k[k.find('<'):(k.find('>')+1)])
    params = list(set(params))
    print('Required Parameters :',params)
    params_info = slab_fields.loc[slab_fields['code'].isin(params)]
    params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
    #print(params_info)
    param_info_owner = params_info[params_info['def_table']=='vt_owner']
    #print('Owner',param_info_owner['column - slab_field'].to_list())
    param_info_other = params_info[params_info['def_table']!='vt_owner']
    #print('Other',param_info_other['column - slab_field'].to_list())
    #param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
    #param_info_misc = params_info[(params_info['table - slab_field']!='vt_tax_based_on_permit_info') & (params_info['table - slab_field']!='vt_tax_based_on')]
    param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
    #print('TBO',param_info_tbo['code'].to_list)
    #print('TBO',param_info_tbo['column - slab_field'].to_list())
    #print('Initial Shape',params_info.shape)
    params_info = extract_parameters(regn_no,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd)
    #print('Final Shape',params_info.shape)                                                                        
    #if pur_cd ==57:        
    print(params_info)
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R':
            #print(c_formula)
            cf_result = eval(c_formula.lower())
        else:
            cf_result = False
        if cf_result==True:
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    #if pur_cd==57:
    #    print(summary_df)
    if len(ind)==0:
        #print('No True Condition')
        return 
    #Module for calculation of rates for true conditions
    true_df = formula_df.loc[ind,:]
    true_df.reset_index(drop=True, inplace=True)
    #print(true_df)
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print(t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        #print(r_formula)
        if 'CEILING' in r_formula: r_formula.replace('CEIL','math.ceil')
        if 'CEIL' in r_formula: r_formula = r_formula.replace('CEIL','math.ceil')
        rate = eval(r_formula.lower())
        rates.append(rate)
        
    true_df['Rate']=rates
    #print(rates)
    #print(true_df)
    #Module for dealing Minimum M and Amount T 
    if len(set(true_df['condition_formula'].to_list()))>1:
        #print('TM',True)
        true_df = true_df[true_df['condition_formula']!='1=1']
        if len({'T','M'} - set(true_df['type_flag'].to_list()))==0:
            if true_df[true_df['type_flag']=='T']['Rate'].to_list()[0]> true_df[true_df['type_flag']=='M']['Rate'].to_list()[0]:
                final_rate = true_df[true_df['type_flag']=='T']['Rate'].to_list()
            else:
                final_rate = true_df[true_df['type_flag']=='M']['Rate'].to_list()
    else:       
        final_rate = true_df['Rate'].to_list()
    if len(final_rate)>0:
        final_rate = final_rate[0]
    else:
        final_rate = 0
        #print(final_rate)
    '''     
    #Module for Tax Mode Remaining years
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        #print('Tax Upto - Tax From =',tax_period,'days')
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        #print(((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days))
        if tax_mode=='Y10':
            days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/3652)*days_rem
        if tax_mode=='Y12':
            days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            #print('Days remaining in ',tax_mode,'=',days_rem,'days')
            final_rate = (final_rate/4383)*days_rem
        if tax_mode=='Y15':
            days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode    
    if tax_mode== 'Q':
        #print(tax_upto,' ',tax_from)
        tax_period = relativedelta(tax_upto, tax_from)
        #print(tax_period)
        years_q = tax_period.years * 4 
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        #print(n_quart)
        final_rate = final_rate * n_quart

    #Display Calculated Amount
    return final_rate

In [43]:
nc_regn_no_list = []
rn = []
oc = []
vc = []
ta = []
tf = []

#print('Sl |','Regn No |','Off_cd |','Vh Class |','Tax Amt-Reprt & Calc |','Tax Fine-Reprt & Calc')
for i in range(9,10):   
    regn_no = def_list['Regn No'].to_list()[i]
    #print(regn_no)
    #state_cd = 'AS'
    off_cd = vt_owner[vt_owner['Regn No']==regn_no]['Off Cd'].to_list()[0]
    vh_class = vt_owner[vt_owner['Regn No']==regn_no]['Vh Class'].to_list()[0]
    
    #rcpt_no = def_list['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    #tax_amt = def_list['Tax Amt'].to_list()[i]
    #rcpt_dt = def_list['Rcpt Dt'].to_list()[i]
    
    #tax_from = def_list['latest_tax_paid_upto'] + 1
    #tax_from = def_list['Tax Arrear From'].to_list()[i]
    tax_from = def_list['Max Upto'].to_list()[i]
    #tax_from = datetime.strptime(tax_from,'%Y-%m-%d')
    #print(tax_from)
    tax_upto = datetime.strptime('31-Jul-2019','%d-%b-%Y')
    #tax_upto = def_list['Tax Arrear UpTo'].to_list()[i]
    #tax_upto = datetime.strptime(tax_upto,'%d-%b-%Y')
    #print(tax_upto)
    c_tax_amt = calculate_tax(regn_no,tax_from,tax_upto)
    if c_tax_amt == 0:
        print('Unable to calculate tax')
        nc_regn_no_list.append(regn_no)
    #tax_amt = pd.to_numeric(def_list['Tax Amount in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    
    #tax_fine = pd.to_numeric(def_list['Tax Fine in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    if c_tax_amt !=0:
        #c_tax_fine = calculate_tax(regn_no,tax_from,tax_upto, pur_cd = 57, final_rate = c_tax_amt,tax_mode ='Y')
        print('')
    else:
        c_tax_fine = 0
    #print(tax_fine)
    #if tax_amt!=c_tax_amt and c_tax_amt != None: 
    print('Mismatch',i,regn_no,'Vh_class',vh_class,c_tax_amt,c_tax_fine)
    #elif tax_amt==c_tax_amt:
    rn.append(regn_no)
    vc.append(vh_class)
    oc.append(off_cd)
    ta.append(c_tax_amt)
    tf.append(c_tax_fine)
def_df = pd.DataFrame(list(zip(rn,oc,vc, ta,tf)), columns =['Regn No','Off Cd','Vh Class','Tax Amt','Tax Fine'])
result = pd.merge(def_df,def_list,on=['Regn No','Off Cd','Vh Class'])
result

70 58 Y 2017-09-30 00:00:00
58 Slab Codes Applicable: [6, 5, 23]
Required Parameters : ['<29>', '<0>', '<25>', '<21>']
   column - slab_field    table_name        column_name  code  \
9            domain_cd  "vt_permit",       "domain_cd",  <21>   
10            seat_cap   "vt_owner",        "seat_cap",   <0>   
17      other_criteria   "vt_owner",  "other_criteria",  <29>   
45         permit_type           NaN        "pmt_type",  <25>   

                                     descr field_type table - slab_field  \
9                 Passenger Vehicle Domain          N    vt_tax_based_on   
10  Seat Cap (Inclusive Driver, Conductor)          N    vt_tax_based_on   
17                          Other Criteria          N    vt_tax_based_on   
45                             Permit Type          N    vt_tax_based_on   

   Clac-YN def_table  def_column  P_value  
9        N       NaN         NaN     12.0  
10       N  vt_owner         NaN      9.0  
17       N  vt_owner         NaN      0.0 

SyntaxError: invalid syntax (<string>, line 1)

In [36]:
def_list = def_list[def_list['Vh Class']==70]
def_list.shape

(943, 23)

In [64]:
result.to_csv(r'D:\AS\New folder\Defaulters Summary Maxi Cab Kamrup Assam.csv')

In [69]:
vt_owner1['Regn Dt'] = vt_owner1['Regn Dt'].astype('datetime64[ns]')

C:\Users\CDMA\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
vt_owner1 = vt_owner[vt_owner['Off Cd']==1]

In [91]:
vt_owner.shape

(3431861, 64)